#### Saturday, January 20, 2024

This all runs.

# Multilingual semantic search

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/04-multilingual.ipynb)

In this example we'll use a multilingual embedding model
[multilingual-e5-base](https://huggingface.co/intfloat/multilingual-e5-base) to perform search on a dataset of mixed
language documents. Using this model, we can search in two ways:
 * Across languages, for example using a query in German to find documents in English
 * Within a non-English language, for example using a query in German to find documents in German

 While this example is using dense retrieval only, it's possible to also combine dense and traditional lexical retrieval
 with hybrid search. For more information on lexical multilingual search, please see the blog post
 [Multilingual search using language identification in Elasticsearch](https://www.elastic.co/search-labs/multilingual-vector-search-e5-embedding-model).

 The dataset used contains snippets of Wikipedia passages from the [MIRACL](https://project-miracl.github.io/) dataset.

# 🧰 Requirements

For this example, you will need:

- Python 3.6 or later
- An Elastic deployment with a machine learning node
   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook))
- The [Elastic Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/installation.html)


## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

Once logged in to your Elastic Cloud account, go to the [Create deployment](https://cloud.elastic.co/deployments/create) page and select **Create deployment**. Leave all settings with their default values.

# Install packages and initialize the Elasticsearch Python client

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the packages we need for this example.

In [ ]:
# !pip install -qU elasticsearch sentence_transformers

Next we need to import the `elasticsearch` module and the `getpass` module.
`getpass` is part of the Python standard library and is used to securely prompt for credentials.

In [1]:
from getpass import getpass
from urllib.request import urlopen
import json
import textwrap
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-base")

# 48m 40.4s

.gitattributes:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Now we can instantiate the Python Elasticsearch client.
First we prompt the user for their password and Cloud ID.

🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [2]:
# # https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
# ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# # https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
# ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# # Create the client instance
# client = Elasticsearch(
#     # For local development
#     # hosts=["http://localhost:9200"] 
#     cloud_id=ELASTIC_CLOUD_ID,
#     api_key=ELASTIC_API_KEY,
# )


# Connect locally ...
esHost = "https://172.19.0.2:9200"
ELASTIC_PASSWORD = "*l-ncw4tr*UF-qzEybkq"
path2cert = "/elasticsearch/http_ca.crt"

client = Elasticsearch(esHost, ca_certs=path2cert, basic_auth=("elastic", ELASTIC_PASSWORD))

Confirm that the client has connected with this test

In [3]:
print(client.info())

{'name': 'f8bb06ea76ae', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'JtIKr7BlTaWWORxN6-qEEw', 'version': {'number': '8.11.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '76013fa76dcbf144c886990c6290715f5dc2ae20', 'build_date': '2023-12-05T10:03:47.729926671Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Create Elasticsearch index with required mappings

We need to add a field to support dense vector storage and search.
Note the `passage_embedding` field below, which is used to store the dense vector representation of the `passage` field.

In [4]:
# Define the mapping
mappings = {
    "properties": {
        "language": {"type": "keyword"},
        "id": {"type": "keyword"},
        "title": {"type": "text"},
        "passage": {"type": "text"},
        "passage_embedding": {
            "type": "dense_vector",
            "dims": 768,
            "index": "true",
            "similarity": "cosine"
        }
    }
}

# Create the index (deleting any previously existing index)
client.indices.delete(index="articles", ignore_unavailable=True)
client.indices.create(index="articles", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'articles'})

## Dataset

Let's index some data.
Note that we are embedding the `passage` field using the sentence transformer model.
Once indexed, you'll see that your documents contain a `passage_embedding` field (`"type": "dense_vector"`) which contains a vector of floating point values.
This is the embedding of the `passage` field in vector space.
We'll use this field to perform semantic search using kNN.

In [5]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/articles.json"
response = urlopen(url)
articles = json.loads(response.read())

## Index documents

Our dataset is a Python list that contains dictionaries of passages from Wikipedia articles in two languages.
We'll use the `client.bulk` method to index our documents in batches.

The following code iterates over the articles and creates a list of actions to be performed.
Each action is a dictionary containing an "index" operation on our Elasticsearch index.
The passage is encoded using our selected model, and the encoded vector is added to the article document.
Note that the E5 models require that a prefix instruction is used "passage: " to tell the model that it is to embed a passage.
On the query side, the query string will be prefixed with "query: ".
The article document is then added to the list of operations.

Finally, we call the `bulk` method, specifying the index name and the list of actions.

In [6]:
operations = []
for article in articles:
    operations.append({"index": {"_index": "articles"}})
    passage = article["passage"]
    passageEmbedding = model.encode(f"passage: {passage}").tolist()
    article["passage_embedding"] = passageEmbedding
    operations.append(article)

client.bulk(index="articles", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 42, 'items': [{'index': {'_index': 'articles', '_id': 'aXziKI0Bw2tVqZnRws1R', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'articles', '_id': 'anziKI0Bw2tVqZnRws1R', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'articles', '_id': 'a3ziKI0Bw2tVqZnRws1R', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'articles', '_id': 'bHziKI0Bw2tVqZnRws1R', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': '

# Multilingual Semantic Search

The `query` function shown below can search for a given text in the dataset, with the query text given in any language. The function supports an optional `language` argument, which when given, restricts the search to passages in the selected language.

In [7]:
def query(q, language=None):
    knn = {
        "field": "passage_embedding",
        "query_vector" : model.encode(f"query: {q}").tolist(),
        "k": 2,
        "num_candidates": 5
    }

    if language:
        knn["filter"] = {
            "term": {
                "language": language,
            }
        }

    return client.search(index="articles", knn=knn)

To make it convenient to experient with this dataset, we will use the following function to format query resposes.

In [8]:
def pretty_response(response):
    if len(response['hits']['hits']) == 0:
        print('Your search returned no results.')
    else:
        for hit in response["hits"]["hits"]:
            score = hit["_score"]
            language = hit["_source"]["language"]
            id = hit["_source"]["id"]
            title = hit["_source"]["title"]
            passage = hit["_source"]["passage"]
            print()
            print(f"ID: {id}")
            print(f"Language: {language}")
            print(f"Title: {title}")
            print(f"Passage: {textwrap.fill(passage, 120)}")
            print(f"Score: {score}")

For those unfamiliar with German, here is a quick translation of search words used in the examples:
 * "health" -> "Gesundheit"
 * "wall" -> "Mauer"

The first example searches for a word in English.

In [9]:
pretty_response(query("health"))


ID: 9002#0
Language: de
Title: Gesundheits- und Krankenpflege
Passage: Die Gesundheits- und Krankenpflege als Berufsfeld umfasst die Versorgung und Betreuung von Menschen aller Altersgruppen,
insbesondere kranke, behinderte und sterbende Erwachsene. Die Gesundheits- und Kinderkrankenpflege hat ihren Schwerpunkt
in der Versorgung von Kindern und Jugendlichen. In beiden Fachrichtungen gehört die Verhütung von Krankheiten und
Gesunderhaltung zum Aufgabengebiet der professionellen Pflege.
Score: 0.8986235

ID: 8881#0
Language: en
Title: Doctor (title)
Passage: Doctor is an academic title that originates from the Latin word of the same spelling and meaning. The word is originally
an agentive noun of the Latin verb "docēre" [dɔˈkeːrɛ] 'to teach'. It has been used as an academic title in Europe since
the 13th century, when the first Doctorates were awarded at the University of Bologna and the University of Paris.
Having become established in European universities, this usage spread around th

Note that in the results above, we see that the document about healthcare,
even though it's in German, matches better to the query "health",
versus the English document which doesn't talk about health specifically but about doctors more generally.
This is the power of a multilingual embedding which embeds meaning across languages.

The next example also searches for a word in English, but only retrieves results in German.

In [10]:
pretty_response(query("wall", language="de"))


ID: 2270104#0
Language: de
Title: London Wall
Passage: London Wall ist die strategische Stadtmauer, die die Römer um Londinium gebaut haben, um die Stadt zu schützen, die über
den wichtigen Hafen an der Themse verfügte. Bis ins späte Mittelalter hinein bildete diese Stadtmauer die Grenzen von
London. Heute ist "London Wall" auch der Name einer Straße, die an einem noch bestehenden Abschnitt der Stadtmauer
verläuft.
Score: 0.89418566

ID: 2270104#1
Language: de
Title: London Wall
Passage: Die Mauer wurde Ende des zweiten oder Anfang des dritten Jahrhunderts erbaut, wahrscheinlich zwischen 190 und 225,
vermutlich zwischen 200 und 220. Sie entstand somit etwa achtzig Jahre nach dem im Jahr 120 erfolgten Bau der Festung,
deren nördliche und westliche Mauern verstärkt und in der Höhe verdoppelt wurden, um einen Teil der neuen Stadtmauer zu
bilden. Die Anlage wurde zumindest bis zum Ende des vierten Jahrhunderts weiter ausgebaut. Sie zählt zu den letzten
großen Bauprojekten der Römer vor de

In the final example, the query is given in German, and only German results are requested.

In [11]:
pretty_response(query("Mauer", language="de"))


ID: 2270104#1
Language: de
Title: London Wall
Passage: Die Mauer wurde Ende des zweiten oder Anfang des dritten Jahrhunderts erbaut, wahrscheinlich zwischen 190 und 225,
vermutlich zwischen 200 und 220. Sie entstand somit etwa achtzig Jahre nach dem im Jahr 120 erfolgten Bau der Festung,
deren nördliche und westliche Mauern verstärkt und in der Höhe verdoppelt wurden, um einen Teil der neuen Stadtmauer zu
bilden. Die Anlage wurde zumindest bis zum Ende des vierten Jahrhunderts weiter ausgebaut. Sie zählt zu den letzten
großen Bauprojekten der Römer vor deren Rückzug aus Britannien im Jahr 410.
Score: 0.88160384

ID: 2270104#0
Language: de
Title: London Wall
Passage: London Wall ist die strategische Stadtmauer, die die Römer um Londinium gebaut haben, um die Stadt zu schützen, die über
den wichtigen Hafen an der Themse verfügte. Bis ins späte Mittelalter hinein bildete diese Stadtmauer die Grenzen von
London. Heute ist "London Wall" auch der Name einer Straße, die an einem noch bestehe